In [1]:
import numpy as np

In [2]:
from pyomo.environ import *
from pyomo.environ import RangeSet
from pyomo.environ import value

In [3]:
A = np.array([
    [1,0,-1,0,0,0,0,0,0],
    [0,1,0,-1,0,0,0,0,0],
    [0,0,1,1,-1,0,0,0.6,0],
    [0,0,0,0,1,-1,0.5,0,0],
    [0,0,0,0,0,1,0,0,0],
    [0,0,0,0,0,1,-1,-1,-1]
])

In [12]:
B = np.array ([
    [1,0.5,1,1,0,0,0.2,0.6,0]
])

In [13]:
C = np.array ([
    [1]
])

In [14]:
coef_GWP=C@B
coef_GWP=np.array(coef_GWP)
coef_GWP = coef_GWP.reshape(-1)

In [15]:
F=1
f=np.zeros(len(A))
f[4]=F

In [16]:
positive_s_indices = []
positive_s_indices = list(range(1, len(np.transpose(A))+1))
#Model Formulation
# Create the model
model = ConcreteModel()

# Define the decision variable
model.set_s = RangeSet(len(np.transpose(A)))
model.s = Var(model.set_s)
model.set_positive_scale = Set(initialize=positive_s_indices)

#model constraint: As = f and s>=0
model.set_balance = RangeSet(len(f))
def balance(model, p): # As = f
    return sum(A[p-1,i-1]*model.s[i] for i in model.set_s) == f[p-1]
def positive_scale(model, i): # s>=0
    return (model.s[i]>=0)

model.balance_constraints = Constraint(model.set_balance, rule=balance)
model.positive_scale_constraints = Constraint(model.set_positive_scale, rule=positive_scale)

model.obj = Objective(expr = sum(coef_GWP[i-1]*model.s[i] for i in model.set_s), sense=minimize)
#Solver
solver = SolverFactory('glpk')
solver.solve(model) # solves and updates instance

{'Problem': [{'Name': 'unknown', 'Lower bound': 0.95, 'Upper bound': 0.95, 'Number of objectives': 1, 'Number of constraints': 16, 'Number of variables': 10, 'Number of nonzeros': 26, 'Sense': 'minimize'}], 'Solver': [{'Status': 'ok', 'Termination condition': 'optimal', 'Statistics': {'Branch and bound': {'Number of bounded subproblems': 0, 'Number of created subproblems': 0}}, 'Error rc': 0, 'Time': 0.04315900802612305}], 'Solution': [OrderedDict([('number of solutions', 0), ('number of solutions displayed', 0)])]}

In [17]:
value(model.obj)

0.95

In [18]:
scaling_factors = []
for j in model.s:
    scaling_factors.append(model.s[j].value)       

In [19]:
scaling_factors

[0.0, 0.5, 0.0, 0.5, 0.5, 1.0, 1.0, 0.0, -0.0]